In [1]:
from numba import cuda

print("CUDA available:", cuda.is_available())

if cuda.is_available():
    device = cuda.get_current_device()
    print("GPU detected:", device.name)
    print("Compute capability:", device.compute_capability)
else:
    print("⚠️ Numba cannot access the GPU.")


CUDA available: True
GPU detected: b'NVIDIA GeForce GTX 1060 6GB'
Compute capability: (6, 1)


In [2]:
from numba import cuda
import numpy as np

@cuda.jit
def add_kernel(a, b, out):
    idx = cuda.grid(1)
    if idx < out.size:
        out[idx] = a[idx] + b[idx]
# Create host arrays
n = 1_000_000
a = np.ones(n, dtype=np.float32)
b = np.ones(n, dtype=np.float32)
out = np.zeros(n, dtype=np.float32)

# Copy to GPU
d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_out = cuda.to_device(out)

# Configure kernel launch
threads_per_block = 256
blocks_per_grid = (n + threads_per_block - 1) // threads_per_block

print(f"Launching kernel: {blocks_per_grid} blocks × {threads_per_block} threads")

# Launch kernel
add_kernel[blocks_per_grid, threads_per_block](d_a, d_b, d_out)

# Copy result back
result = d_out.copy_to_host()

print("Done!")
import numpy as np

if np.all(result == 2):
    print("✅ SUCCESS — GPU computed correct result.")
else:
    print("❌ ERROR — incorrect output.")
    print(result[:10])


Launching kernel: 3907 blocks × 256 threads
Done!
✅ SUCCESS — GPU computed correct result.


In [3]:
pip freeze

anyio==4.12.0
argon2-cffi==25.1.0
argon2-cffi-bindings==25.1.0
arrow==1.4.0
asttokens==3.0.1
async-lru==2.0.5
attrs==25.4.0
babel==2.17.0
beautifulsoup4==4.14.3
bleach==6.3.0
certifi==2025.11.12
cffi==2.0.0
charset-normalizer==3.4.4
comm==0.2.3
contourpy==1.3.3
cuda-bindings==12.9.4
cuda-pathfinder==1.3.3
cuda-python==12.9.0
cupy-cuda13x==13.6.0
cycler==0.12.1
Cython==3.2.2
debugpy==1.8.17
decorator==5.2.1
defusedxml==0.7.1
executing==2.2.1
fastjsonschema==2.21.2
fastrlock==0.8.3
fonttools==4.61.0
fqdn==1.5.1
h11==0.16.0
httpcore==1.0.9
httpx==0.28.1
idna==3.11
ImageIO==2.37.2
ipykernel==7.1.0
ipython==9.8.0
ipython_pygments_lexers==1.1.1
ipywidgets==8.1.8
isoduration==20.11.0
jedi==0.19.2
Jinja2==3.1.6
joblib==1.5.2
json5==0.12.1
jsonpointer==3.0.0
jsonschema==4.25.1
jsonschema-specifications==2025.9.1
jupyter-events==0.12.0
jupyter-lsp==2.3.0
jupyter_client==8.6.3
jupyter_core==5.9.1
jupyter_server==2.17.0
jupyter_server_terminals==0.5.3
jupyterlab==4.5.0
jupyterlab_pygments==0.3.0
j